# potential projects:
+ Fuel consumption 2000-2022
https://www.kaggle.com/datasets/ahmettyilmazz/fuel-consumption/data
  + https://www.kaggle.com/code/arpitasinha12/fuel-consumption-eda


+ UFO Sightings Around The World
https://www.kaggle.com/datasets/jonwright13/ufo-sightings-around-the-world-better/code
  + on maps: https://www.kaggle.com/code/mikedelong/ufo-sightings-on-maps
  + EDA: https://www.kaggle.com/code/veetikoivuniemi/ufo-sightings-around-the-world-eda






